In [1]:
## Dependency libraries used for the RRG:
import numpy as np
import networkx as nx
import scipy as spy
import itertools
import copy
from scipy.signal import argrelextrema
from scipy.linalg import expm

In [2]:
def HighOrderLaplician(A, Type, Order):
    ## Input: 
    # A is the 1st-order adjacency matrix of the graph

    # Type is a str that determines which type of high order Laplician representation to use
    # Type='MOL': the function generates the multi-order Laplacian operator, which is 
    # Type='HOPL': the function generates the high-order path Laplacian operator proposed in our work

    # Order is a number that determines which order of interactions to analyze

    ## Output:
    # L is the high order Laplician representation, which is the Multiorder Laplacian operator or the high-order path Laplacian
    Num = A.shape[0]
    # Stores the vertices
    store = [0]* (Order+1)

    # Degree of the vertices
    # d = [deg for (_, deg) in G.degree()]
    d = list(np.sum(A,axis=0))

    Cliques = []
    # Function to check if the given set of vertices
    # in store array is a clique or not
    def is_clique(b) :
        # Run a loop for all the set of edges
        # for the select vertex
        for i in range(b) :
            for j in range(i + 1, b) :
                # If any edge is missing
                if (A[store[i]][store[j]] == 0) :
                    return False
        return True

    # Function to find all the cliques of size s
    def findCliques(i, l, s) :    
        # Check if any vertex from i+1 can be inserted as the l-th node in the simplex of size s
        for j in range( i + 1, Num) :
            # If the degree of the graph is sufficient
            if (d[j] >= s - 1) :
                # Add the vertex to store
                store[l] = j
                # If the graph is not a clique of size k
                # then it cannot be a clique
                # by adding another edge
                if (is_clique(l + 1)) :
                    # If the length of the clique is
                    # still less than the desired size
                    if (l < s-1) :
                        # Recursion to add vertices
                        findCliques(j, l + 1, s)
                    # Size is met
                    else :
                        Cliques.append(store[:s])
    findCliques(-1, 0, Order+1)

    if len(Cliques)>0:
        if Type=='MOL': # the multi-order Laplacian operator
            HO_A = np.zeros((Num, Num))
            for Simplex in Cliques:
                for [i, j] in itertools.combinations(Simplex,2):
                    HO_A[i,j] += 1
            HO_A += HO_A.T
            HO_D = sum(HO_A)
            L = Order * np.diag(HO_D) - HO_A
        elif Type=='HOPL': # the high-order path Laplacian operator
            HO_A = np.zeros((Num, Num))
            for Simplex in Cliques:
                for [i, j] in itertools.combinations(Simplex,2):
                    HO_A[i,j] += 1
            HO_A += HO_A.T
            HO_D = sum(HO_A)
            L = np.diag(HO_D) - HO_A   
    else:
        L=np.zeros((Num,Num))
    return L

def NetworkInfo(L):
    ## Input: 
    # L is the high order Laplician representation, which can be the Multiorder Laplacian operator or the high-order path Laplacian

    ## Output:
    # Sub_Ls the list of the sub-Laplician-matrices associated with all connected components from L
    # Sub_NodeIDs is the list of sub-node-indice associated with all connected components from L
    G=nx.from_numpy_array(np.abs(L)-np.diag(np.diag(L)))
    Sub_Ls=[L[list(SG),:][:,list(SG)] for SG in nx.connected_components(G)] 
    Sub_NodeIDs=[list(SG) for SG in nx.connected_components(G)]
    return Sub_Ls, Sub_NodeIDs

def NetworkUnion(Sub_Ls):
    L=np.ones((1,1))
    for SL in Sub_Ls:
        L=spy.linalg.block_diag(L,SL)
    L=L[1:,1:]
    return L

def SRG_Flow(G,q,p,L_Type,IterNum):
    A=nx.adjacency_matrix(G).toarray()       
    L=HighOrderLaplician(A, L_Type, Order=q)
    L0=HighOrderLaplician(A, L_Type, Order=p)
    L_List,L0_List,C_List,Tracked_Alignment=SRG_Function(L,L0,q,IterNum)
    return L_List,L0_List,C_List,Tracked_Alignment

    
def SRG_Function(L,L0,q,IterNum):
    ## Input: 
    # L is the initial guiding high order Laplician representation, which can be the Multiorder Laplacian operator or the high-order path Laplacian
    # L0 is the initial guided high order Laplician representation, which can be the Multiorder Laplacian operator or the high-order path Laplacian

    ## Output:
    # L_List is the list of the guiding q-order Laplacian over renormalization steps
    # L0_List is the list of the guided p-order Laplacian over renormalization steps
    # C_List is the list of specific heat vector calculated by the initial q - order Laplacian L_List[0] and a range of time scale
    # Tracked_Alignment is the indexes of the initial units aggregated into each macro-unit of all connected clusters after every iteration of the SRG
    Iter=1
    Init_Sub_Ls,_=NetworkInfo(L)
    L_List=[L]
    L0_List=[L0]
    tau_Vec=np.zeros(len(Init_Sub_Ls))
    C_List=[]
    for ID in range(len(Init_Sub_Ls)):
        if np.size(Init_Sub_Ls[ID],0)>1:
            tau,CVector=TauSelection(Init_Sub_Ls[ID])
            tau=tau/((q+1)/2)
            tau_Vec[ID]=tau
            C_List.append(CVector)
    All_Alignment=[]
    while Iter<IterNum:
        Current_L=L_List[Iter-1]
        Current_L0=L0_List[Iter-1]
        Current_G0=nx.from_numpy_array(np.abs(Current_L0)-np.diag(np.diag(Current_L0)))
        Sub_Ls, Sub_NodeIDs=NetworkInfo(Current_L)
        NewSub_Ls=[]
        ClusterNodeAlignment=[]
        for SLID in range(len(Sub_Ls)):
            SL=Sub_Ls[SLID]
            SNodeID = Sub_NodeIDs[SLID]
            if np.size(SL,0)==1:
                NodeAlignment=[]
                NewSub_Ls.append(SL)
                NodeAlignment.append([SNodeID[0],SNodeID])
            else:
                NodeAlignment=[]
                Evals, _ = np.linalg.eig(SL)
                tau=tau_Vec[SLID]
                Eval_idx = np.where(np.real(Evals)>1/tau)[0]

                Rho = expm(-tau*SL)/np.trace(expm(-tau*SL))
                rho_Evals, rho_Evecs = np.linalg.eig(Rho)
                rho = np.zeros((Rho.shape[0],Rho.shape[1]))
                for ID in Eval_idx:
                    Evec = rho_Evecs[:,ID].reshape(-1,1)
                    rho += np.real(rho_Evals[ID]*np.matmul(Evec,Evec.T))
                adj1 = np.zeros((rho.shape[0],rho.shape[1]))
                for i in range(SL.shape[0]):
                    for j in range(SL.shape[1]):
                        if i==j:
                            continue
                        elif ((rho[i][j]>=rho[j][j]) or (rho[i][j]>=rho[i][i])):
                            adj1[i][j]=1
                            adj1[j][i]=1
                RefG = nx.from_numpy_array(adj1) # reference graph
                # delete false edges not in subgraph of SL
                Current_SG=nx.from_numpy_array(np.abs(SL)-np.diag(np.diag(SL)))
                Potential_Clusters=[list(c) for c in list(nx.connected_components(RefG))]
                Edge_To_Remove=[]
                for ID1 in range(len(Potential_Clusters)):
                    Unit_list=Potential_Clusters[ID1]
                    if len(Unit_list)>1:
                        H = nx.induced_subgraph(Current_SG,Unit_list)
                        Potential_H = nx.induced_subgraph(RefG,Unit_list)
                        Wrong_Edge=list(set(list(Potential_H.edges))-set(list(H.edges)))
                        Edge_To_Remove.extend(Wrong_Edge)
                for Wrong_Edge in Edge_To_Remove:
                    RefG.remove_edge(*Wrong_Edge)
                Clusters=[list(c) for c in list(nx.connected_components(RefG))]
                # contracting SG nodes
                for Nodes in Clusters:
                    Node1 = Nodes[0]
                    for node in Nodes[1:]:
                        Current_SG = nx.contracted_nodes(Current_SG, Node1, node, self_loops=False)
                mapping = {old_label:new_label for new_label, old_label in enumerate(Current_SG.nodes())}
                Current_SG = nx.relabel_nodes(Current_SG, mapping)
                New_SA = nx.adjacency_matrix(Current_SG).toarray()
                # contracting G0 nodes
                for Nodes in Clusters:
                    SNodes = [SNodeID[node] for node in Nodes]
                    Node1 = SNodes[0]
                    NodeAlignment.append([Node1, SNodes])
                    for node in SNodes[1:]:
                        Current_G0 = nx.contracted_nodes(Current_G0, Node1, node, self_loops=False)
                    
                New_SL = np.diag(np.sum(New_SA,axis=0)) - New_SA
                NewSub_Ls.append(New_SL)
            ClusterNodeAlignment.append(NodeAlignment)
        All_Alignment.append(ClusterNodeAlignment)

        mapping = {old_label:new_label for new_label, old_label in enumerate(Current_G0.nodes())}
        Current_G0 = nx.relabel_nodes(Current_G0, mapping)
        New_A0 = nx.adjacency_matrix(Current_G0).toarray()
        New_L0 = np.diag(np.sum(New_A0,axis=0)) - New_A0
        New_L=NetworkUnion(NewSub_Ls)
        print('Nn:', New_L.shape[0])

        L_List.append(New_L)
        L0_List.append(New_L0)
        Iter=Iter+1
    Tracked_Alignment=TrackingUnitID(All_Alignment,L.shape[0])
    return L_List,L0_List,C_List,Tracked_Alignment

def TauSelection(L):
    ## Input: 
    # L is the initial high order Laplician representation, which can be the Multiorder Laplacian operator or the high-order path Laplacian

    ## Output:
    # tau is the ideal constant that determines the time scale
    TauVec=np.logspace(-2, 2, 200) # reconmennd tau in (1e-2,1e0) for q>1
    MLambdaVector=np.zeros_like(TauVec)
    for ID in range(len(TauVec)):
        Poss_tau=TauVec[ID]
        MatrixExp=spy.linalg.expm(-Poss_tau*L)
        Rho=MatrixExp/np.trace(MatrixExp)
        MLambdaVector[ID]=np.trace(L @ Rho)
    CVector=-np.power(TauVec[1:],2)*np.diff(MLambdaVector)/np.diff(TauVec)
    CVector = CVector[np.where(~np.isnan(CVector))[0]]
    TauVec = TauVec[np.where(~np.isnan(CVector))[0]]

    potential_localmax_id = argrelextrema(CVector,np.greater)[0]
    true_localmax_id = potential_localmax_id[np.where(CVector[potential_localmax_id]>0.5*np.max(CVector))[0]]
    tau=TauVec[true_localmax_id[0]]

    dCVector=np.diff(CVector)/np.diff(TauVec)
    plateau_idx = np.where((np.abs(dCVector)<5e-2)&(TauVec[1:]>=tau))[0]
    plateau_idx = [i for i in plateau_idx if i+1 in plateau_idx] # find consecutive index as true plateau
    if len(plateau_idx)>50 and (CVector[plateau_idx]>0.1*np.max(CVector)).all():
        tau = TauVec[plateau_idx[0]]
    print(['our method: tau is ',tau,'!!'])
    return tau,CVector

def TrackingUnitID(All_Alignment,UnitNum):
    # convert nodeID within an iteration (All_Alignment) to global nodeID (Tracked_Alignment)
    Tracked_Alignment=copy.deepcopy(All_Alignment)
    UnitIDVec=list(range(UnitNum))
    for IterID in range(len(All_Alignment)):
        if IterID>0:
            for ClusterID in range(len(All_Alignment[IterID])):
                for CoarseID in range(len(All_Alignment[IterID][ClusterID])):
                    NodesToTrack=All_Alignment[IterID][ClusterID][CoarseID][1]
                    for IDT in range(len(NodesToTrack)):
                        TrackedID=UnitIDVec[NodesToTrack[IDT]]
                        Tracked_Alignment[IterID][ClusterID][CoarseID][1][IDT]=TrackedID
                    Tracked_Alignment[IterID][ClusterID][CoarseID][0]=Tracked_Alignment[IterID][ClusterID][CoarseID][1][0]
        # delete Coarsed node
        NodetoDelete=[]
        for ClusterID in range(len(Tracked_Alignment[IterID])):
            for CoarseID in range(len(Tracked_Alignment[IterID][ClusterID])):
                Nodes=Tracked_Alignment[IterID][ClusterID][CoarseID][1][1:]
                NodetoDelete.extend(Nodes)

        for IDD in NodetoDelete:
            UnitIDVec.remove(IDD)

    return Tracked_Alignment

In [3]:
G=nx.random_graphs.barabasi_albert_graph(1000,4) # Generate a a random BA network with 1000 units

In [ ]:
# Multiorder Laplacian operator
L_List,L0_List,C_List,Tracked_Alignment=SRG_Flow(G,q=1,p=1,L_Type='MOL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 1-order interactions

L_List,L0_List,C_List,Tracked_Alignment=SRG_Flow(G,q=2,p=1,L_Type='MOL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 2-order interactions

L_List,L0_List,C_List,Tracked_Alignment=SRG_Flow(G,q=3,p=1,L_Type='MOL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 3-order interactions

In [ ]:
# High-order path Laplacian
L_List,L0_List,C_List,Tracked_Alignment=SRG_Flow(G,q=1,p=1,L_Type='HOPL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 1-order interactions

L_List,L0_List,C_List,Tracked_Alignment=SRG_Flow(G,q=2,p=1,L_Type='HOPL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 2-order interactions

L_List,L0_List,C_List,Tracked_Alignment=SRG_Flow(G,q=3,p=1,L_Type='HOPL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 3-order interactions